In [ ]:
# Cell 1 — Install deps (if needed) & GPU check
!pip install -q torch torchvision pillow matplotlib scikit-learn

import torch, sys
print("Python:", sys.version.splitlines()[0])
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("Device:", torch.cuda.get_device_name(0))
    except:
        print("CUDA available but device name fetch failed.")
else:
    print("No GPU detected. Make sure runtime -> Change runtime type -> GPU.")
# Cell 2 — Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Drive mounted at /content/drive")
# Step 4 — CLEAN dataloaders (ImageFolder or CIFAR fallback)
import os, random, numpy as np
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

BATCH_SIZE = 32
NUM_WORKERS = 2
DATASET_PATH = "/content/drive/MyDrive/dataset"   # change if needed

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.15,0.15,0.15,0.02),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def get_dataloaders(path=DATASET_PATH, batch_size=BATCH_SIZE):
    train_path = Path(path) / "train"
    val_path   = Path(path) / "val"

    if train_path.exists() and val_path.exists():
        print("🔥 Using ImageFolder dataset from:", path)
        train_ds = datasets.ImageFolder(str(train_path), transform=train_tf)
        val_ds   = datasets.ImageFolder(str(val_path),   transform=val_tf)
        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
        val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS)
        return train_loader, val_loader, train_ds.classes

    print("⚠️  ImageFolder dataset not found. Falling back to CIFAR10 (cat/dog/car).")
    cifar_tf = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
    cifar_train = datasets.CIFAR10(root="/content/cifar", train=True, download=True, transform=cifar_tf)
    cifar_test  = datasets.CIFAR10(root="/content/cifar", train=False, download=True, transform=cifar_tf)

    def is_kept(label):
        return label in [3,5,1,9]

    train_idx = [i for i, (_,y) in enumerate(cifar_train) if is_kept(y)][:2000]
    test_idx  = [i for i, (_,y) in enumerate(cifar_test)  if is_kept(y)][:1000]

    def remap_label(lbl):
        if lbl == 3: return 0   # cat
        if lbl == 5: return 1   # dog
        return 2                # car

    class CIFARFiltered(torch.utils.data.Dataset):
        def __init__(self, base_ds, idxs):
            self.base = base_ds
            self.idxs = idxs
        def __len__(self): return len(self.idxs)
        def __getitem__(self, i):
            x, lbl = self.base[self.idxs[i]]
            return x, remap_label(lbl)

    train_ds = CIFARFiltered(cifar_train, train_idx)
    val_ds   = CIFARFiltered(cifar_test,  test_idx)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=NUM_WORKERS)
    val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=NUM_WORKERS)

    return train_loader, val_loader, ["cat","dog","car"]

# Run to create variables
train_loader, val_loader, classes = get_dataloaders()
print("\nClasses:", classes)
print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))
xb, yb = next(iter(train_loader))
print("Sample batch shape:", xb.shape, yb.shape)
# STEP 5 — Build Model (ResNet18 fine-tuning)

import torch
import torch.nn as nn
from torchvision import models

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

def build_model(num_classes, freeze_backbone=False):
    # Load pretrained weights (new PyTorch 2.0 API)
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

    # If you want faster training at first: freeze backbone
    if freeze_backbone:
        for name, param in model.named_parameters():
            if "fc" not in name:
                param.requires_grad = False

    # Replace final layer for our number of classes
    model.fc = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(model.fc.in_features, num_classes)
    )

    return model.to(DEVICE)

print("Classes detected:", classes)   # must exist from Step 4

# Build the model
model = build_model(len(classes))

print("\nModel created successfully!")
print("Output classes:", len(classes))
# STEP 6 — Train the model and save best checkpoint

import torch
import torch.nn as nn
import torch.nn.functional as F
import time, copy

NUM_EPOCHS = 8         # you can increase later
LR = 1e-3
SAVE_PATH = "/content/drive/MyDrive/best_imagetagger.pth"   # model save location

def train_model(model, train_loader, val_loader, classes, epochs=NUM_EPOCHS, lr=LR):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

    best_acc = 0.0
    best_wts = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        print(f"\n🚀 Epoch {epoch+1}/{epochs}")

        # -------------------
        # TRAINING PHASE
        # -------------------
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        for x, y in train_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            preds = output.argmax(1)
            total_loss += loss.item() * x.size(0)
            total_correct += (preds == y).sum().item()
            total_samples += x.size(0)

        train_acc = total_correct / total_samples
        train_loss = total_loss / total_samples

        # -------------------
        # VALIDATION PHASE
        # -------------------
        model.eval()
        val_loss = 0
        val_correct = 0
        val_samples = 0

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                output = model(x)
                loss = criterion(output, y)
                preds = output.argmax(1)

                val_loss += loss.item() * x.size(0)
                val_correct += (preds == y).sum().item()
                val_samples += x.size(0)

        val_acc = val_correct / val_samples
        val_loss = val_loss / val_samples

        print(f"   Train Acc: {train_acc:.4f}   |   Val Acc: {val_acc:.4f}")

        # SAVE BEST MODEL
        if val_acc > best_acc:
            best_acc = val_acc
            best_wts = copy.deepcopy(model.state_dict())
            torch.save({"model_state": best_wts, "classes": classes}, SAVE_PATH)
            print(f"   💾 Saved new best model! ({SAVE_PATH})")

        scheduler.step()

    print("\n✅ Training complete!")
    print("Best Validation Accuracy:", best_acc)

    # load best weights before returning
    model.load_state_dict(best_wts)
    return model


# -------- RUN TRAINING --------
model = train_model(model, train_loader, val_loader, classes)
# STEP 7 — Evaluation + confusion matrix + show examples
# Run after training (expects: model or saved checkpoint, val_loader, classes, DEVICE)

import os
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import pandas as pd

# Paths
CKPT_PATH = "/content/drive/MyDrive/best_imagetagger.pth"
OUT_DIR = "/content/drive/MyDrive/imagetagger_eval"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) load the best checkpoint if present (otherwise use current model variable)
if os.path.exists(CKPT_PATH):
    print("Loading checkpoint from:", CKPT_PATH)
    ck = torch.load(CKPT_PATH, map_location=DEVICE)
    ck_classes = ck.get("classes", None)
    if ck_classes is not None and ck_classes != classes:
        print("Warning: checkpoint classes differ from current `classes` variable. Using checkpoint's classes.")
        classes = ck_classes
    # build model of matching size and load weights (safe load)
    try:
        temp_model = model  # try to reuse existing model variable shape
        temp_model.load_state_dict(ck["model_state"])
        temp_model.eval()
        eval_model = temp_model
    except Exception as e:
        # fallback: rebuild then load
        from torchvision import models
        import torch.nn as nn
        eval_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        eval_model.fc = torch.nn.Sequential(torch.nn.Dropout(0.4), torch.nn.Linear(eval_model.fc.in_features, len(classes)))
        eval_model.load_state_dict(ck["model_state"])
        eval_model = eval_model.to(DEVICE)
else:
    print("No checkpoint found at", CKPT_PATH, " — using current `model` in memory.")
    eval_model = model

eval_model.eval()

# 2) run through validation set and collect preds & probs
all_preds = []
all_labels = []
all_probs = []   # top1 prob for each sample
sample_images = []  # store some examples for display (pil images + meta)

with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(DEVICE)
        out = eval_model(xb)
        probs = F.softmax(out, dim=1)
        top1 = probs.argmax(1).cpu().numpy().tolist()
        top1p = probs.max(1)[0].cpu().numpy().tolist()
        all_preds.extend(top1)
        all_labels.extend(yb.numpy().tolist())
        all_probs.extend(top1p)
        # save a few images and their preds for display (convert from tensor)
        for i in range(min(3, xb.size(0))):
            # inverse normalize for display
            t = xb[i].cpu()
            inv_norm = torch.tensor([[-0.485/0.229, -0.456/0.224, -0.406/0.225],
                                     [1/0.229, 1/0.224, 1/0.225]])
            mean = torch.tensor([0.485,0.456,0.406]).view(3,1,1)
            std  = torch.tensor([0.229,0.224,0.225]).view(3,1,1)
            img = t * std + mean
            img = (img.clamp(0,1).permute(1,2,0).numpy() * 255).astype(np.uint8)
            sample_images.append({
                "pil": Image.fromarray(img),
                "true": int(yb[i].item()),
                "pred": int(top1[i]),
                "prob": float(top1p[i])
            })

# 3) classification report + confusion matrix
print("\n== Classification report ==")
report = classification_report(all_labels, all_preds, target_names=classes, digits=4, output_dict=True)
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion matrix:\n", cm)

# Save report to CSV/JSON
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(os.path.join(OUT_DIR, "classification_report.csv"))
pd.DataFrame(cm, index=classes, columns=classes).to_csv(os.path.join(OUT_DIR, "confusion_matrix.csv"))
print("\nSaved evaluation CSVs to:", OUT_DIR)

# 4) plot confusion matrix
plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation="nearest", cmap="Blues")
plt.title("Confusion matrix")
plt.colorbar()
ticks = np.arange(len(classes))
plt.xticks(ticks, classes, rotation=45)
plt.yticks(ticks, classes)
plt.xlabel("Predicted")
plt.ylabel("True")
for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, cm[i,j], ha="center", va="center", color="white" if cm[i,j] > cm.max()/2 else "black")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "confusion_matrix.png"))
plt.show()

# 5) show some correct & misclassified examples (up to 8)
print("\n== Sample examples (true -> pred : prob) ==")
show_count = 0
fig = plt.figure(figsize=(12,6))
for item in sample_images[:8]:
    ax = fig.add_subplot(2,4,show_count+1)
    ax.imshow(item["pil"])
    title = f"{classes[item['true']]} → {classes[item['pred']]} : {item['prob']:.2f}"
    ax.set_title(title)
    ax.axis("off")
    show_count += 1
plt.tight_layout()
plt.show()

# 6) summary print and done
acc_overall = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"\nOverall validation accuracy: {acc_overall:.4f}")
print("Saved files in:", OUT_DIR)
!ls -lh /content/drive/MyDrive
IMG_PATH = "/content/drive/MyDrive/car1.png"
# or
IMG_PATH = "/content/drive/MyDrive/myphoto.jpg"
# List all images in MyDrive
import glob

files = glob.glob("/content/drive/MyDrive/*.*")

print("Files in MyDrive:")
for f in files:
    print(f)
IMG_PATH = "/content/drive/MyDrive/car.jpeg"
# Auto-pick latest image in MyDrive and predict (single cell)
import glob, os, torch
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn as nn

DRIVE_ROOT = "/content/drive/MyDrive"
CKPT_PATH = "/content/drive/MyDrive/best_imagetagger.pth"
IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp", ".tiff"]

# 1) find latest image file in MyDrive
candidates = []
for ext in IMG_EXTS:
    candidates += glob.glob(os.path.join(DRIVE_ROOT, f"*{ext}"))
    candidates += glob.glob(os.path.join(DRIVE_ROOT, f"**/*{ext}"), recursive=True)
candidates = list(set(candidates))  # unique

if not candidates:
    raise FileNotFoundError(f"No image files found under {DRIVE_ROOT}. Drop an image there (MyDrive root) and re-run.")

# choose most recently modified
latest = max(candidates, key=os.path.getmtime)
print("Using latest image:", latest)

# 2) check checkpoint
if not os.path.exists(CKPT_PATH):
    raise FileNotFoundError(f"Model checkpoint not found at {CKPT_PATH}. Run training cell to create it.")

# 3) load checkpoint and rebuild model safely
ck = torch.load(CKPT_PATH, map_location="cpu")
classes = ck.get("classes", None)
if classes is None:
    raise RuntimeError("Checkpoint missing 'classes'. Can't rebuild model.")

# rebuild model
model_pred = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model_pred.fc = nn.Sequential(nn.Dropout(0.4), nn.Linear(model_pred.fc.in_features, len(classes)))
model_pred.load_state_dict(ck["model_state"])
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_pred = model_pred.to(DEVICE).eval()

# 4) prepare transform (re-define val_tf lightly to ensure availability)
from torchvision import transforms
val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# 5) predict
img = Image.open(latest).convert("RGB")
x = val_tf(img).unsqueeze(0).to(DEVICE)
with torch.no_grad():
    out = model_pred(x)
    probs = F.softmax(out, dim=1)[0]
    top_p, top_i = probs.topk(min(3, len(classes)))
results = [(classes[i], float(top_p[j])) for j,i in enumerate(top_i)]

# 6) show results
print("\n🔥 Top predictions (class → confidence):")
for cls, p in results:
    print(f"  {cls} → {p:.4f}")

plt.figure(figsize=(6,6))
plt.imshow(img)
plt.axis("off")
plt.title(f"Predicted: {results[0][0]} ({results[0][1]:.2f})")
plt.show()
